To work on colab, we need to add some paths and install libraries that are already installed on local computer. This part is not needed to run on local machine.



In [ ]:
!pip install patchify
!pip install dropbox
!pip install gputil
!pip install psutil
!pip install humanize

We need to test the given GPU statistics

In [ ]:
# Import packages
import os,sys,humanize,psutil,GPUtil

# Define function
def mem_report():
  print("CPU RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ))
  
  GPUs = GPUtil.getGPUs()
  for i, gpu in enumerate(GPUs):
    print('GPU {:d} ... Mem Free: {:.0f}MB / {:.0f}MB | Utilization {:3.0f}%'.format(i, gpu.memoryFree, gpu.memoryTotal, gpu.memoryUtil*100))
    
# Execute function
mem_report()

## Define Paths and Folders

Define the paths and folders for the training and testing.

In [ ]:
import os
dropboxFolderPath = 'E:/Dropbox/'

# define the paths relative to the dropbox folder
unetWorkingPath = os.path.join(dropboxFolderPath, 'Education/PhD/Projects/MucilageDetection/uNetLearning')

sentinelTestImagePath = '/Dataset/satellite/sentinel2/35TPE_MATDATA/'

In [ ]:
import sys
from google.colab import drive

# mount the drive
drive.mount('/content/drive', force_remount=True)

# override the local paths
unetWorkingPath = '/content/drive/Othercomputers/My Laptop/uNetLearning/'

# add the paths to system paths 
sys.path.append(unetWorkingPath)
sys.path.append(os.path.join(unetWorkingPath, 'functions'))

# Prepare For Train and Test

Training code has been written in python. We use uNet architecture to segment the mucilage from the water. The implementation fulfilled with pyTorch. *italicized text*

In [ ]:
# define common paths
modelSavePath = os.path.join(unetWorkingPath, 'models')
sentinelTrainImagePath = os.path.join(unetWorkingPath, 'patches')

## Import Libraries

To utilize training, we need to import necessary libraries. Some of the libraries are the standart pyTorch libarries and these can be imported via colab. The custom libraries are imported via the google drive.

In [ ]:
# these are the standart libraries
import os
import torch
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchsummary import summary

# these are the custom libraries and will be imported from the drive
from unet.unet_model import UNet
from functions.ModelTrainer import train_model
from functions.SentinelLoader import SentinelPatchLoader
from functions.DataTransformer import GetDataTransformer

## Create Device

In [ ]:
# create pytorch device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Training device is {device}')

# clear GPU memory
if device == 'cuda':
  torch.cuda.empty_cache()
  torch.cuda.clear_cache()

## Define Optional Settings

We need to set the necessary parameters for training. The path for the training and validation patches are automatically selected as local path or drive path.

batchSize: set the mini batch size for the training
patchSize: number of patches that are cropped from the training images
loadAllAtOnce: load all data to memory before running, this will increase the training process but memory could be insufficient

In [ ]:
# options
patchSize = 192
batchSize = 64
patchCount = 200
loadAllAtOnce = True

## Train and Validation Dataset

In [ ]:
# print the location of patches
print(f'Sentinel Patches will be used from {sentinelTrainImagePath}')

# define train images
TrainingImages = ["S2A_MSIL2A_20210402T085551_N0300_R007_T35TPF_20210402T133128",
                  "S2A_MSIL2A_20210509T084601_N0300_R107_T35TPF_20210509T115513",
                  "S2A_MSIL2A_20210512T085601_N0300_R007_T35TPF_20210512T133202"]
TrainingDataLoader = SentinelPatchLoader(sentinelTrainImagePath, TrainingImages, patchCount, loadAllAtOnce, GetDataTransformer())

# define validation images
ValidationImages = ["S2B_MSIL2A_20210414T084559_N0300_R107_T35TPF_20210414T112733"]
ValidationDataLoader = SentinelPatchLoader(sentinelTrainImagePath, ValidationImages, patchCount, loadAllAtOnce, GetDataTransformer())

In [ ]:
# create custom data loader
TrainingDataLoader = {
    'train': DataLoader(TrainingDataLoader, batch_size=batchSize, shuffle=True, num_workers=0),
    'val': DataLoader(ValidationDataLoader, batch_size=batchSize, shuffle=True, num_workers=0)
}

## Create Model

Create the uNet model which has 9 channel input and 1 semantic class. After creating the model we will try to import the previous best weights for initialization. If the previous training run interrupted, code will try to recover from the last known state by loading the trainedModel. 

In [ ]:
# create a ResNetUNet model and apply it to model
trainModel = UNet(n_channels=9, n_classes=1)
trainModel = trainModel.to(device)

# print the model summary
summary(trainModel, (9, patchSize, patchSize))

# define optimizer and scheduler
optimizer = optim.Adam(filter(lambda p: p.requires_grad, trainModel.parameters()), lr=0.0005)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.5)

# check for previous model
lastTrainedModelPath = os.path.join(modelSavePath, 'trainedModel')
preTrainedModelPath = os.path.join(modelSavePath,'bestModel')
if os.path.isfile(lastTrainedModelPath):
    print('loading the previous model...')
    
    # load the model
    checkpoint = torch.load(lastTrainedModelPath)
    trainModel.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']
elif os.path.isfile(preTrainedModelPath):
    print('using the pre-trained model weights...')
    weights = torch.load(preTrainedModelPath)
    trainModel.load_state_dict(weights)
else:
    print('no previous model found, training from scratch')

# Training Code

Training code has been written in python. We use uNet architecture to segment the mucilage from the water. The implementation fulfilled with pyTorch.

## Define Train and Test Set

In this part we define **train** and **validation** set. Since we label only 4 images, we use 3 of them for training and 1 of them for validation. 

*Note that this part will take some time if the loadAllAtOnce flag set to True, but it will speed up the traing*

In [ ]:
# start training
print('training model...')
train_model(trainModel, optimizer, exp_lr_scheduler, TrainingDataLoader, device, num_epochs=200, outputPath=modelSavePath)

# Test Model

In this section we are going to use the model trained in the previous section and generate the output for different images.

In [ ]:
TestingImages = {
    "S2A_MSIL2A_20210102T090351_N0214_R007_T35TPE_20210102T115327_20m.mat",
    "S2A_MSIL2A_20210119T085301_N0214_R107_T35TPE_20210119T115427_20m.mat",
    "S2A_MSIL2A_20210129T085221_N0214_R107_T35TPE_20210129T103011_20m.mat",
    "S2A_MSIL2A_20210221T090001_N0214_R007_T35TPE_20210221T115031_20m.mat",
    "S2A_MSIL2A_20210303T085851_N0214_R007_T35TPE_20210303T105606_20m.mat",
    "S2A_MSIL2A_20210313T085741_N0214_R007_T35TPE_20210313T112414_20m.mat",
    "S2A_MSIL2A_20210402T085551_N0300_R007_T35TPE_20210402T133128_20m.mat",
    "S2A_MSIL2A_20210422T085551_N0300_R007_T35TPE_20210422T122938_20m.mat",
    "S2A_MSIL2A_20210429T084601_N0300_R107_T35TPE_20210429T103912_20m.mat",
    "S2A_MSIL2A_20210509T084601_N0300_R107_T35TPE_20210509T115513_20m.mat",
    "S2A_MSIL2A_20210512T085601_N0300_R007_T35TPE_20210512T133202_20m.mat",
    "S2A_MSIL2A_20210519T084601_N0300_R107_T35TPE_20210519T115101_20m.mat",
    "S2A_MSIL2A_20210519T084601_N0300_R107_T35TPF_20210519T115101_20m.mat",
    "S2A_MSIL2A_20210611T085601_N0300_R007_T35TPE_20210611T121904_20m.mat",
    "S2A_MSIL2A_20210618T084601_N0300_R107_T35TPE_20210618T120513_20m.mat",
    "S2A_MSIL2A_20210628T084601_N0300_R107_T35TPE_20210628T103709_20m.mat",
    "S2A_MSIL2A_20210701T085601_N0301_R007_T35TPE_20210701T125029_20m.mat",
    "S2A_MSIL2A_20210711T085601_N0301_R007_T35TPE_20210711T121659_20m.mat",
    "S2A_MSIL2A_20210718T084601_N0301_R107_T35TPE_20210718T114908_20m.mat",
    "S2A_MSIL2A_20210728T084601_N0301_R107_T35TPE_20210728T111949_20m.mat",
    "S2A_MSIL2A_20210731T085601_N0301_R007_T35TPE_20210731T120834_20m.mat",
    "S2A_MSIL2A_20210807T084601_N0301_R107_T35TPE_20210807T114132_20m.mat",
    "S2A_MSIL2A_20210810T085601_N0301_R007_T35TPE_20210810T120517_20m.mat",
    "S2A_MSIL2A_20210817T084601_N0301_R107_T35TPE_20210817T114824_20m.mat",
    "S2A_MSIL2A_20210820T085601_N0301_R007_T35TPE_20210820T122214_20m.mat",
    "S2A_MSIL2A_20210827T084601_N0301_R107_T35TPE_20210827T102802_20m.mat",
    "S2A_MSIL2A_20210830T085601_N0301_R007_T35TPE_20210831T164139_20m.mat",
    "S2B_MSIL2A_20210203T085059_N0214_R107_T35TPE_20210203T111110_20m.mat",
    "S2B_MSIL2A_20210206T090039_N0214_R007_T35TPE_20210206T112619_20m.mat",
    "S2B_MSIL2A_20210223T084849_N0214_R107_T35TPE_20210223T111955_20m.mat",
    "S2B_MSIL2A_20210226T085829_N0214_R007_T35TPE_20210226T112525_20m.mat",
    "S2B_MSIL2A_20210305T084739_N0214_R107_T35TPE_20210305T111205_20m.mat",
    "S2B_MSIL2A_20210308T085719_N0214_R007_T35TPE_20210308T113808_20m.mat",
    "S2B_MSIL2A_20210328T085559_N0214_R007_T35TPE_20210328T113525_20m.mat",
    "S2B_MSIL2A_20210407T085549_N0300_R007_T35TPE_20210407T123314_20m.mat",
    "S2B_MSIL2A_20210414T084559_N0300_R107_T35TPE_20210414T112733_20m.mat",
    "S2B_MSIL2A_20210427T085549_N0300_R007_T35TPE_20210427T113845_20m.mat",
    "S2B_MSIL2A_20210507T085559_N0300_R007_T35TPE_20210507T192808_20m.mat",
    "S2B_MSIL2A_20210514T084559_N0300_R107_T35TPE_20210514T113538_20m.mat",
    "S2B_MSIL2A_20210517T085559_N0300_R007_T35TPE_20210517T112912_20m.mat",
    "S2B_MSIL2A_20210517T085559_N0300_R007_T35TPF_20210517T112912_20m.mat",
    "S2B_MSIL2A_20210524T084559_N0300_R107_T35TPE_20210524T111238_20m.mat",
    "S2B_MSIL2A_20210606T085559_N0300_R007_T35TPE_20210606T120423_20m.mat",
    "S2B_MSIL2A_20210613T084559_N0300_R107_T35TPE_20210613T113603_20m.mat",
    "S2B_MSIL2A_20210623T084559_N0300_R107_T35TPE_20210623T112759_20m.mat",
    "S2B_MSIL2A_20210626T085559_N0300_R007_T35TPE_20210626T114028_20m.mat",
    "S2B_MSIL2A_20210713T084559_N0301_R107_T35TPE_20210713T115731_20m.mat",
    "S2B_MSIL2A_20210716T085559_N0301_R007_T35TPE_20210716T112700_20m.mat",
    "S2B_MSIL2A_20210726T085559_N0301_R007_T35TPE_20210726T122123_20m.mat",
    "S2B_MSIL2A_20210802T084559_N0301_R107_T35TPE_20210802T110537_20m.mat",
    "S2B_MSIL2A_20210805T085559_N0301_R007_T35TPE_20210805T113028_20m.mat",
    "S2B_MSIL2A_20210812T084559_N0301_R107_T35TPE_20210812T105857_20m.mat",
    "S2B_MSIL2A_20210815T085559_N0301_R007_T35TPE_20210815T111606_20m.mat",
    "S2B_MSIL2A_20210822T084559_N0301_R107_T35TPE_20210822T110737_20m.mat",
    "S2B_MSIL2A_20210825T085559_N0301_R007_T35TPE_20210825T120947_20m.mat",
}

## Options

Set the test options

- modelResolution is needed for output array
- testbatchSize can be larger than the trainBatchSize
- cropZone is used to reduce tested patch size

In [ ]:
testBatchSize = 64
cropZone = (0,0, 4223, 1727)

## Create Model and Load the Best Weights

Create the same model as in training phase and load the model parameters from the training.

In [ ]:
# include test related libraries
import numpy as np
import torch.nn.functional as func
import scipy.io as sio

from functions.SentinelLoader import SentinelTestDataset
from functions.TestImagePathFinder import GetTestImagePath

In [ ]:
testModel = UNet(n_channels=9, n_classes=1)
testModel = testModel.to(device)
print('loading pretrained model...')
testModel.load_state_dict(torch.load(os.path.join(modelSavePath, 'bestModel')))

In [ ]:
# create output directory
outputDirectory = os.path.join(unetWorkingPath, "outputs")
if not os.path.exists(outputDirectory):
    os.makedirs(outputDirectory)

In [ ]:
# test the images one by one and save the result
for TestImage in TestingImages:
    
    # get the path to the image
    ImagePath = GetTestImagePath(dropboxFolderPath, sentinelTestImagePath, TestImage)

    # create loader
    TestingDataLoader = SentinelTestDataset(ImagePath, cropZone, patchSize, GetDataTransformer())
    
    # load the patches with batchSize
    TestDataLoader = {
        'test': DataLoader(TestingDataLoader, batch_size=testBatchSize, shuffle=False, num_workers=0)
    }
    
    # create output patches
    patchCountX = (cropZone[2] - cropZone[0]) // patchSize
    patchCountY = (cropZone[3] - cropZone[1]) // patchSize
    result = np.zeros((patchCountY,patchCountX,1,patchSize,patchSize), dtype=np.float32)
    
    # find the results
    for inputs, rows, cols in TestDataLoader['test']:
        inputs = inputs.to(device)
    
        # get the result
        with torch.set_grad_enabled(False):
            outputs = func.sigmoid(testModel(inputs)).contiguous()
    
        # convert tensor to np array
        outputNP = outputs.cpu().detach().numpy()
        rowsNP = rows.cpu().detach().numpy()
        colsNP = cols.cpu().detach().numpy()
        
        # add the result into the big array
        for i in range(outputNP.shape[0]):
            result[rowsNP[i],colsNP[i],:,:,:] = outputNP[i,:,:,:]
        
    # convert patches to image
    OutputFileName = os.path.splitext(TestImage)[0]+'_MUCILAGE.mat'
    sio.savemat(os.path.join(outputDirectory, OutputFileName), {'mucilage': result})